In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
import glob

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat

--2025-10-06 13:40:01--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz [following]
--2025-10-06 13:40:01--  https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/octet-stream]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  26.3MB/s    in 13s     

2025-10-06 13:40:15 (25.1 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2025-10-06 13:40:15--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels

In [ ]:
!mkdir -p data
!mv imagelabels.mat data/

In [ ]:
!tar -xzf 102flowers.tgz

In [ ]:
img_labels_mat = scipy.io.loadmat('data/imagelabels.mat')
targets = img_labels_mat['labels'][0]
targets = np.array(targets) - 1

In [ ]:
image_paths = sorted(glob.glob('jpg/*.jpg'))
total_examples = len(image_paths)

In [ ]:
path_label_dataset = tf.data.Dataset.from_tensor_slices((image_paths, targets))
path_label_dataset = path_label_dataset.shuffle(buffer_size=total_examples, seed=42)

In [ ]:
train_size = int(0.8 * total_examples)
validation_size = int(0.1 * total_examples)

In [ ]:
ds_train = path_label_dataset.take(train_size)
ds_validation = path_label_dataset.skip(train_size).take(validation_size)
ds_test = path_label_dataset.skip(train_size + validation_size)

In [ ]:
img_size = (128, 128)
batch_size = 32
num_classes = 102
autotune = tf.data.AUTOTUNE

def parse_image(filepath, label):
    image = tf.io.read_file(filepath)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, img_size)
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_dataset = ds_train.map(parse_image, num_parallel_calls=autotune).cache().batch(batch_size).prefetch(autotune)
validation_dataset = ds_validation.map(parse_image, num_parallel_calls=autotune).cache().batch(batch_size).prefetch(autotune)
test_dataset = ds_test.map(parse_image, num_parallel_calls=autotune).cache().batch(batch_size).prefetch(autotune)

In [ ]:
from tensorflow.keras import layers
basic_model = tf.keras.Sequential([
    layers.Input(shape=img_size + (3,)),
    layers.Rescaling(1./255),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(128, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation="softmax")
])



In [ ]:
basic_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=3,
    restore_best_weights=True)

basic_model.fit(train_dataset,
    validation_data=validation_dataset,
    callbacks=[early_stopping],
    epochs=20)

Epoch 1/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.0518 - loss: 4.3594 - val_accuracy: 0.1528 - val_loss: 3.5447
Epoch 2/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.1618 - loss: 3.4638 - val_accuracy: 0.3411 - val_loss: 2.6225
Epoch 3/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2899 - loss: 2.7996 - val_accuracy: 0.5122 - val_loss: 1.9415
Epoch 4/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.4140 - loss: 2.2387 - val_accuracy: 0.6650 - val_loss: 1.3934
Epoch 5/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5233 - loss: 1.7446 - val_accuracy: 0.7042 - val_loss: 1.1723
Epoch 6/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6263 - loss: 1.3121 - val_accuracy: 0.7971 - val_loss: 0.8339
Epoch 7/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7047 - loss: 1.0049 - val_accuracy: 0.8411 - val_loss: 0.6758
Epoch 8/20
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7771 - loss: 0.7291 - val_acc

In [ ]:
loss, accuracy = basic_model.evaluate(validation_dataset)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy*100:.2f}%")

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9315 - loss: 0.3488
Validation Loss: 0.3781
Validation Accuracy: 92.42%
